# Woolsey Fire Data Exploration

In [ ]:
import os
from pathlib import Path

import earthpy.plot as ep
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
from rasterio.enums import Resampling
import rioxarray as rxr
import xarray as xr

from ea_drought_burn.config import CRS, DATA_DIR
from ea_drought_burn.utils import (
    open_raster,
    plot_bands,
    plot_rgb,
    reproject_match
)




BOUND_KWARGS = {
    "facecolor": "none",
    "edgecolor": "black",
    "linewidth": 2
}
PRISM_CMAP = "summer_r"




# User may need to update path
os.chdir(DATA_DIR)

In [ ]:
def plot_four(bands, mask, *bounds, width=16, **kwargs):
    """Plots bands in two columns"""
    
    # Scale fonts
    font_size = plt.rcParamsDefault["font.size"]
    plt.rc("font", size=font_size * width // 16)
    
    # Scale outline
    bound_kwargs = BOUND_KWARGS.copy()
    bound_kwargs["linewidth"] *= width // 16
    
    titles = [mask.format(year) for year in range(2013, 2017)]
    
    nrows = int(np.ceil(len(bands) / 2))
    height = (width * 0.7) / 2 * nrows

    fig, axes = plt.subplots(nrows, 2, figsize=(width, height))
    
    axes_flat = []
    for row in axes:
        try:
            axes_flat.extend(row)
        except TypeError:
            axes_flat.append(row)

    for ax, title, band in zip(axes_flat, titles, bands):
        plot_bands(band, ax=ax, title=title, **kwargs)
        for bound in bounds:
            bound.plot(ax=ax, **bound_kwargs)
    
    # Reset font size to default
    plt.rcParams.update(plt.rcParamsDefault)

# Woolsey Fire Boundary

In [ ]:
# Load the Woolsey Fire perimeter
bbox = [-119.3, 33.9, -118.2, 34.5]
woolsey_fire = load_nifc_fires("2018-CAVNC-091023", crs=CRS, bbox=bbox)

# Set crop boundary
crop_bound = woolsey_fire.envelope

# Set fire boundaries to draw on map
related_fires = woolsey_fire

In [ ]:
# SMM stack is used for reprojections so load here
# FIXME: SMM stack is loaded twice
smm_stack = open_raster(
    os.path.join("data", "SMMDroughtstack.dat"),
    crs=CRS,
    crop_bound=crop_bound
)

# Sentinel pre- and post-fire images

In [ ]:
# Load pre-fire data from Sentinel
path = os.path.join("custom",
                    "sentinel_2",
                    "3_aligned",
                    "L1C_T11SLT_A008633_20181031T184032.tif")
prefire = open_raster(path, crs=CRS)

plot_rgb(prefire, rgb=[3, 2, 1], stretch=True)

In [ ]:
# Load post-fire data from Sentinel
path = os.path.join("custom",
                    "sentinel_2",
                    "3_aligned",
                    "L1C_T11SLT_A018185_20181215T184316.tif")
postfire = open_raster(path, crs=CRS)

plot_rgb(postfire, rgb=[3, 2, 1], stretch=True)

In [ ]:
# Calculate NBR
nir = prefire[7]
swir2 = prefire[11]
pre_nbr = (nir - swir2) / (nir + swir2)
#plot_bands(pre_nbr, cmap="RdYlGn", vmin=-1, vmax=1)

nir = postfire[7]
swir2 = postfire[11]
post_nbr = (nir - swir2) / (nir + swir2)
#plot_bands(post_nbr, cmap="RdYlGn", vmin=-1, vmax=1)

dnbr = pre_nbr - post_nbr

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr,
           ax=ax,
           cmap="RdYlGn_r",
           title="Sentinel NBR - Woolsey Fire\n Oct 31, 2018 - Dec 15, 2018")
related_fires.plot(ax=ax, **BOUND_KWARGS)

In [ ]:
# Define dNBR classification bins
dnbr_class_bins = [-np.inf, -.1, .1, .27, .66, np.inf]
dnbr_classified = xr.apply_ufunc(np.digitize, dnbr, dnbr_class_bins)

dnbr_classified = dnbr_classified.astype(np.float64)
dnbr_classified = dnbr_classified.rio.clip(woolsey_fire.geometry)

dnbr_cat_names = [
    "Increased Greenness",
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
]

nbr_colors = [
    "#7FFF00",
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
]
nbr_cmap = ListedColormap(nbr_colors)

# Plot the data with a custom legend
fig, ax = plt.subplots(figsize=(10, 8))

plot_bands(dnbr_classified,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title="Sentinel dNBR - Woolsey Fire\n Oct 31, 2018 - Dec 15, 2018",
           cbar=False,
           scale=False,
           ax=ax)
related_fires.plot(ax=ax, **BOUND_KWARGS)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names)

In [ ]:
# MTBS dNBR
path = os.path.join("custom",
                    "mtbs",
                    "2018",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr.tif")
dnbr = open_raster(path, crs=CRS)
dnbr = dnbr.where(dnbr != -9999, np.nan)
dnbr = reproject_match(dnbr, smm_stack[0])

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr,
           ax=ax,
           cmap="RdYlGn_r",
           title="MTBS dNBR - Woolsey Fire\nDec 15, 2017 - Dec 15, 2018")
related_fires.plot(ax=ax, **BOUND_KWARGS)

In [ ]:
# MTBS classified dNBR

path = os.path.join("custom",
                    "mtbs",
                    "2018",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr6.tif")
dnbr_classified = open_raster(path, crs=CRS, crop_bound=woolsey_fire.geometry)
dnbr_classified = dnbr_classified.where(dnbr_classified != -9999, np.nan)

dnbr_classified = reproject_match(dnbr_classified, smm_stack[0])

dnbr_cat_names = [
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
    "Increased Greenness"
]

nbr_colors = [
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
]
nbr_cmap = ListedColormap(nbr_colors)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr_classified,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("MTBS classified dNBR - Woolsey Fire\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False,
           ax=ax)
related_fires.plot(ax=ax, **BOUND_KWARGS)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names,
               bbox=(0.01, 0.99))

In [ ]:
# Define dNBR classification bins
dnbr_class_bins = [-np.inf, 100, 400, 800, 1100, np.inf]
dnbr_classified = xr.apply_ufunc(np.digitize, dnbr, dnbr_class_bins)

dnbr_classified = dnbr_classified.astype(np.float64)
dnbr_classified = dnbr_classified.rio.clip(woolsey_fire.geometry)

dnbr_cat_names = [
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
    "Increased Greenness"
]

nbr_colors = [
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
]
nbr_cmap = ListedColormap(nbr_colors)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr_classified,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("MTBS classified dNBR - Woolsey Fire\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False,
           ax=ax)
related_fires.plot(ax=ax, **BOUND_KWARGS)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names,
               bbox=(0.01, 0.99))

# Landsat pre- and post-fire images

In [ ]:
# Load pre-fire data from Sentinel
path = os.path.join("custom",
                    "landsat_8",
                    "3_aligned",
                    "LC080410362018110301T1-SC20210422052513.tif")
prefire = open_raster(path, crs=CRS)

plot_rgb(prefire, rgb=[3, 2, 1], stretch=True)

In [ ]:
# Load post-fire data from Sentinel
path = os.path.join("custom",
                    "landsat_8",
                    "3_aligned",
                    "LC080410362018111901T1-SC20181203132746.tif")
postfire = open_raster(path, crs=CRS)

plot_rgb(postfire, rgb=[3, 2, 1], stretch=True)

In [ ]:
# No idea what Landsat dNBR file is
path = os.path.join("data",
                    "Fires",
                    "WoolseyFire",
                    "Burn Severity",
                    "Landsat",
                    "dNBR.tif")
dnbr = reproject_match(open_raster(path, crs=CRS), smm_stack[0])

plot_bands(dnbr, cmap="RdYlGn", vmin=-1, vmax=1)

# Woolsey Fire Stack

The Woolsey Fire stack also includes FAL and community data, but that is repeated
in the SMM stack below.

In [ ]:
woolsey_stack = open_raster(
    os.path.join("data", "Fires", "WoolseyFire", "WoolseyFireDataStack2"),
    crs=CRS,
    crop_bound=crop_bound,
    nodata=np.nan
)

woolsey_stack

## Landsat sprectal index

The WoolseyFireDataStack2 file contains two sets of Landsat files, one from
2017-10-24 and one from 2018-10-18. Both pre-date the Woolsey fire and include
only bands 4 (red) and 7 (SWIR2). None of the spectral indices I'm aware of use
those two bands.

## BARC

BARC stands for Burned Area Reflectance Classification. It bins fire damage
into four categories: high, moderate, low, and unburned. In the image below,
the bins use the following colors:

+ Unburned: (0, 100, 0)
+ Low: (128, 255, 210)
+ Moderate: (255, 255, 0)
+ High: (255, 100, 0)

In [ ]:
# Plot BARC
barc = woolsey_stack[4:7]

fig, ax = plt.subplots(figsize=(12, 12))
plot_rgb(barc, ax=ax, title="BARC")
related_fires.plot(ax=ax, **BOUND_KWARGS)

# Approximate colors used in the plot
colors = [(0, 100, 0), (128, 255, 210), (255, 255, 0), (255, 100, 0)]
im_ax = ax.get_images()[0]
im_ax.cmap = ListedColormap([[c / 255 for c in c] for c in colors])

# Draw legend
labels = ["Unburned", "Low", "Moderate", "High"]
ep.draw_legend(im_ax, 
               titles=labels,
               classes=range(len(labels)))

# SMM Stack

The nodata value for the weather/climate data in the SMM stack is -3.40282306e+38.
This value is not set in the file, so the data needs to be masked manually.

In [ ]:
smm_stack = open_raster(
    os.path.join("data", "SMMDroughtstack.dat"),
    crs=CRS,
    crop_bound=crop_bound
)

smm_stack

In [ ]:
# Plot vegetation community
smm_community = smm_stack[0]

colors = [
    "black",
    "tab:green",
    "tab:olive",
    "tab:cyan",
    "tab:red",
    "tab:blue",
    "lightgray"
]

font_size = plt.rcParamsDefault["font.size"]
plt.rc("font", size=font_size * 2)

fig, ax = plt.subplots(figsize=(20, 20))
plot_bands(smm_community,
           ax=ax,
           title="Vegetation community map",
           cmap=ListedColormap(colors),
           cbar=False)

bound_kwargs = BOUND_KWARGS.copy()
bound_kwargs["linewidth"] *= 2
related_fires.plot(ax=ax, **bound_kwargs)

# Draw legend
labels = [
    "No data",
    "Annual grass",
    "Chaparral",
    "Coastal sage scrub",
    "Oak woodland",
    "Riparian",
    "Substrate"
]
ep.draw_legend(ax.get_images()[0], 
               titles=labels,
               classes=range(len(labels)))

plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
# Plot fraction alive
fal = smm_stack[1:5].copy()

# The nodata value for the FAL plots is -3.40282306e+38, but that is not set
# on the source file. All other values are between 0 and 1.
fal = fal.where(fal >= 0, np.nan)

# Resampling nodata set to np.nan yields an all white map!
#fal = fal.rio.reproject(fal.rio.crs, 4048, resampling=Resampling.nearest)

plot_four(fal,
          "Fraction alive ({})",
          related_fires,
          width=64,
          cmap="PiYG",
          vmin=0,
          vmax=1)

In [ ]:
# Plot year-on-year difference in fraction alive

dfal_13_14 = fal[0] - fal[1]
dfal_14_15 = fal[1] - fal[2]
dfal_15_16 = fal[2] - fal[3]
dfal_13_16 = fal[0] - fal[3]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(64, 48))

font_size = plt.rcParamsDefault["font.size"]
plt.rc("font", size=font_size * 4)

plot_bands(dfal_13_14, ax=ax1, cmap="PiYG", vmin=-1, vmax=1, title="dFAL (2013-2014)")
plot_bands(dfal_14_15, ax=ax2, cmap="PiYG", vmin=-1, vmax=1, title="dFAL (2014-2015)")
plot_bands(dfal_15_16, ax=ax3, cmap="PiYG", vmin=-1, vmax=1, title="dFAL (2015-2016)")
plot_bands(dfal_13_16, ax=ax4, cmap="PiYG", vmin=-1, vmax=1, title="dFAL (2013-2016)")

bound_kwargs = BOUND_KWARGS.copy()
bound_kwargs["linewidth"] *= 4
for ax in (ax1, ax2, ax3, ax4):
    related_fires.plot(ax=ax, **bound_kwargs)

plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
# Plot number of days with precipiation
days_precip = smm_stack[5:9]
days_precip = days_precip.where(days_precip > 0, np.nan)

plot_four(days_precip, 
          "Days precipitation ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=days_precip.min(),
          vmax=days_precip.max())

In [ ]:
# Plot maximum VPD
max_vpd = smm_stack[9:13]
max_vpd = max_vpd.where(max_vpd >= 0, np.nan)

plot_four(max_vpd, 
          "Max VPD ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=max_vpd.min(),
          vmax=max_vpd.max())

In [ ]:
# Plot minimum temperatue
min_temp = smm_stack[13:17]
min_temp = min_temp.where(min_temp > 0, np.nan)

plot_four(min_temp, 
          "Minimum temperature ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=min_temp.min(),
          vmax=min_temp.max())

In [ ]:
# Plot number of days over 95 F
heat_days_over_95 = smm_stack[17:21]
heat_days_over_95 = heat_days_over_95.where(heat_days_over_95 >= 0, np.nan)

plot_four(heat_days_over_95, 
          "Heat days over 95 ({})",
          related_fires,
          cmap="winter",
          vmin=heat_days_over_95.min(),
          vmax=heat_days_over_95.max())

In [ ]:
# Plot cumulative precipitation
cumulative_precip = smm_stack[21:25]
cumulative_precip = cumulative_precip.where(cumulative_precip >= 0, np.nan)

plot_four(cumulative_precip, 
          "Cumulative precipitation ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=cumulative_precip.min(),
          vmax=cumulative_precip.max())

In [ ]:
# Calculate area lost per vegetation type
# FIXME: Check this, numbers differ from previous work

print(f"Total area:   {2089 * 1681 * 15.6 **2 / 1e6:.0f} km2")
print("--")

comm = smm_stack[0]
for i in range(0, 7):
    subset = xr.where(comm == i, True, False).values
    color = colors[i]
    
    # Limit to vegetation (i.e., pixels with FAL calculated)
    comm_13_16 = dfal_13_16.where(subset)
    
    dfal = comm_13_16.mean().item()
    pixels = xr.where(comm_13_16, 1, 0).sum().item()
    pixel_km2 = comm_13_16.rio.resolution()[0] ** 2 * 1e-6
    
    area_km2 = dfal * pixels * pixel_km2
    
    print(labels[i])
    print(f"Pixels:          {pixels}")
    print(f"Area (km2):      {pixels * pixel_km2:.1f} km2")
    print(f"Area (acres):    {pixels * pixel_km2 / 0.004:.1f} acres")
    print(f"Dieback %:       {dfal:.3f}")
    print(f"Dieback (km2):   {area_km2:.1f} km2")
    print(f"Dieback (acres): {area_km2 / 0.004:.1f} acres")
    print("--")
    
    """fig, ax = plt.subplots(figsize=(32, 32))
    plot_bands(dfal_13_16,
               ax=ax,
               title=f"Vegetation community {i} map",
               cmap="PiYG",
               cbar=True)

    bound_kwargs = BOUND_KWARGS.copy()
    bound_kwargs["linewidth"] *= 1
    related_fires.plot(ax=ax, **bound_kwargs)"""


In [ ]:
# Plot dead pixels
half_dead = xr.where(fal < 0.5, True, False)
half_dead = half_dead.rio.clip(woolsey_fire.geometry)
half_dead = half_dead.where(half_dead != -2147483648, False)

plot_four(half_dead,
          ">50% dead ({})",
          related_fires,
          width=64,
          cmap="Greys",
          cbar=False)